# 📊 Notebook 04: Val/Test DataLoaders

**Purpose:** Create DataLoaders for validation and test sets with `shuffle=False` and larger batch sizes.

**What you'll learn:** How evaluation data loading differs from training.


## 🎯 Concept Primer: Evaluation DataLoaders

### Key Differences: Training vs Evaluation

| Parameter | Training | Validation | Test |
|-----------|----------|------------|------|
| `shuffle` | ✅ True | ❌ False | ❌ False |
| `batch_size` | Small (8-16) | Large (32-64) | Large (32-64) |
| `drop_last` | Optional | ❌ False | ❌ False |

### Why shuffle=False for Evaluation?
- Consistent, repeatable metrics
- Easier debugging (same order every time)
- No benefit from shuffling (no training happens)

### Why Larger Batch Sizes for Evaluation?
- **No backpropagation** → less memory needed
- **Faster inference** → fewer iterations
- Batch size of 32 or 64 is common

### drop_last Consideration
- `drop_last=True`: Drops the last incomplete batch
- **Training:** Sometimes used to keep batch sizes consistent
- **Evaluation:** Always `False` (we want to evaluate ALL samples!)

**Example:** 100 samples, batch_size=32
- Batches: [32, 32, 32, 4]
- If `drop_last=True`, we'd skip 4 samples → biased metrics!


## 📚 Learning Objectives

By the end of this notebook, you will:

1. ✅ Create `val_dataset` and `test_dataset` with `val_test_transform`
2. ✅ Create `val_dataloader` and `test_dataloader` with `shuffle=False`
3. ✅ Use `batch_size=32` for faster evaluation
4. ✅ Verify shapes: `images=[32,3,96,96]`, `labels=[32]` (or smaller for last batch)
5. ✅ Understand why evaluation doesn't shuffle


## ✅ Acceptance Criteria

Your val/test data loaders are correct when:

- [ ] `val_dataset` and `test_dataset` use `val_test_transform`
- [ ] `val_dataloader` and `test_dataloader` have `shuffle=False`
- [ ] Both use `batch_size=32`
- [ ] Iterating one batch produces `images.shape = [32, 3, 96, 96]` (or less for last batch)
- [ ] Running the iteration twice yields batches in the **same order**


---

## 💻 TODO 1: Import Libraries & Rebuild val_test_transform

**What you need:**
- PyTorch, DataLoader, transforms
- `PCamDataset`
- Rebuild `val_test_transform` from Notebook 03


In [ ]:
# TODO 1: Import libraries and rebuild val_test_transform
# Hint: import torch
# Hint: from torch.utils.data import DataLoader
# Hint: from torchvision import transforms
# Hint: from src.datasets.pcam_dataset import PCamDataset

# YOUR CODE HERE

# Rebuild val_test_transform (copy from Notebook 03)
val_test_transform = None  # Replace with transforms.Compose([...])

print("✅ Imports successful and val_test_transform ready")


---

## 💻 TODO 2: Create Validation & Test Datasets


In [ ]:
# TODO 2: Create val_dataset and test_dataset
# Hint: val_dataset = PCamDataset(csv_file='../data/validation_labels.csv', transform=val_test_transform)
# Hint: test_dataset = PCamDataset(csv_file='../data/test_labels.csv', transform=val_test_transform)

# YOUR CODE HERE
val_dataset = None  # Replace this line
test_dataset = None  # Replace this line

print(f"✅ Validation dataset: {len(val_dataset)} samples")
print(f"✅ Test dataset: {len(test_dataset)} samples")


---

## 💻 TODO 3: Create Validation & Test DataLoaders (shuffle=False!)


In [ ]:
# TODO 3: Create val_dataloader and test_dataloader
# Hint: DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)

# YOUR CODE HERE
val_dataloader = None  # Replace this line
test_dataloader = None  # Replace this line

print(f"✅ Validation DataLoader: {len(val_dataloader)} batches")
print(f"✅ Test DataLoader: {len(test_dataloader)} batches")


---

## 💻 TODO 4: Test the Val DataLoader


In [ ]:
# TODO 4: Iterate one batch from val_dataloader
# Hint: for images, labels in val_dataloader: ...

# YOUR CODE HERE

print("✅ First validation batch loaded")
# Print images.shape, labels.shape


---

## 🤔 Reflection Prompts

### Question 1: Why Larger Batch Sizes for Evaluation?
Training uses `batch_size=8`, but evaluation uses `batch_size=32`.

**Question:** What allows us to use larger batches during evaluation?

**Your answer:**

---

### Question 2: Consistency Check
Run this code twice:

```python
first_batch_labels_run1 = next(iter(val_dataloader))[1]
first_batch_labels_run2 = next(iter(val_dataloader))[1]
```

**Questions:**
- Will `first_batch_labels_run1` equal `first_batch_labels_run2`?
- Why or why not?
- What if `val_dataloader` had `shuffle=True`?

**Your analysis:**

---


## 🚀 Next Steps

Excellent! You now have all three data loaders ready.

**Move to Notebook 05:** Simple CNN Architecture

**Key Takeaway:** Train=shuffle, Val/Test=no shuffle + larger batches!
